In [21]:
import glob
fn = glob.glob('/home/huangzhengjie/quora_pair/quora_stack/hyperopt_output/train/*.npy')

In [2]:
import re
def pattern(x, f1, f2):
    if re.search(f1, x) is None:
        return False
    if re.search(f2, x) is None:
        return False
    return True
for n in [wd for wd in fn if pattern(wd, 'xgboost_gblinear','\[zj\]')]:
    print n.split('/')[-1]
    loss = float(n.split('/')[-1].split('-')[-2])
    print loss

In [28]:
p1 = [ 'xgboost', 'nn', 'logistic_regression']
p2 = ['\[zj\]', '\[zjb\]', '\[wxh\]']
bst = []
for pp in p1:
    for pp2 in p2:
        da = []
        for n in [wd for wd in fn if pattern(wd, pp, pp2)]:
            name = n.split('/')[-1]
            loss = float(n.split('/')[-1].split('-')[-2])
            da.append((name, loss))
        da = sorted(da, lambda x, y: cmp(x[1], y[1]))[:5]
        bst.append((pp+'-'+pp2[2:-2],da))


In [29]:
import numpy as np
def bagging(fn_list):
    xtrs = None
    xtes = None
    for fn in fn_list:
        xtr = np.load('/home/huangzhengjie/quora_pair/quora_stack/hyperopt_output/train/' + fn[0])
        xte = np.load('/home/huangzhengjie/quora_pair/quora_stack/hyperopt_output/test/' + fn[0])
        if xtrs is None:
            xtrs = xtr
        else:
            xtrs +=  xtr
        if xtes is None:
            xtes = xte
        else:
            xtes += xte
    xtrs /= len(fn_list)
    xtes /= len(fn_list)
    return xtrs, xtes

In [30]:
x_train_mat = []
x_test_mat = []
col = []
for i in range(len(bst)):
    print bst[i][0]
    print bst[i][1][0][1]
    x1, x2 = bagging(bst[i][1])
    if len(x1.shape) > 1:
        x1 = x1.squeeze()
    if len(x2.shape) > 1:
        x2 = x2.squeeze()
    x_train_mat.append(x1)
    x_test_mat.append(x2)
    col.append(bst[i][0])

xgboost-zj
0.186552862244
xgboost-zjb
0.182150746352
xgboost-wxh
0.190170920451
nn-zj
0.186916765399
nn-zjb
0.183799792651
nn-wxh
0.192653776264
logistic_regression-zj
0.197837593982
logistic_regression-zjb
0.210405496108
logistic_regression-wxh
0.217911928804


In [31]:
import pandas as pd
train_pd = pd.read_csv('../../train.csv')

In [32]:
for xid, cl in enumerate(col):
    train_pd[str(cl)] = x_train_mat[xid]

In [33]:
train_pd[col].corr()

,xgboost-zj,xgboost-zjb,xgboost-wxh,nn-zj,nn-zjb,nn-wxh,logistic_regression-zj,logistic_regression-zjb,logistic_regression-wxh
xgboost-zj,1.000000,0.974998,0.974963,0.996694,0.973875,0.974116,0.991721,0.960871,0.959425
xgboost-zjb,0.974998,1.000000,0.979895,0.974377,0.994338,0.977561,0.968239,0.977915,0.959678
xgboost-wxh,0.974963,0.979895,1.000000,0.974101,0.977655,0.994081,0.968430,0.962330,0.977146
nn-zj,0.996694,0.974377,0.974101,1.000000,0.973922,0.973867,0.992219,0.960513,0.958813
nn-zjb,0.973875,0.994338,0.977655,0.973922,1.000000,0.979184,0.967489,0.981084,0.961872
nn-wxh,0.974116,0.977561,0.994081,0.973867,0.979184,1.000000,0.967783,0.964808,0.980090
logistic_regression-zj,0.991721,0.968239,0.968430,0.992219,0.967489,0.967783,1.000000,0.960254,0.958841
logistic_regression-zjb,0.960871,0.977915,0.962330,0.960513,0.981084,0.964808,0.960254,1.000000,0.972207
logistic_regression-wxh,0.959425,0.959678,0.977146,0.958813,0.961872,0.980090,0.958841,0.972207,1.000000


In [34]:
for col_id, col_name in enumerate(col):
    np.save('/home/huangzhengjie/quora_pair/quora_stack/hyperopt_output_stage2/train/' + col_name +'.npy', x_train_mat[col_id])
    np.save('/home/huangzhengjie/quora_pair/quora_stack/hyperopt_output_stage2/test/' + col_name +'.npy', x_test_mat[col_id])

In [35]:
x_tr = np.concatenate([np.expand_dims(wd, -1) for wd in x_train_mat], axis=-1)
x_te = np.concatenate([np.expand_dims(wd, -1) for wd in x_test_mat], axis=-1)

In [37]:
np.save('/data1/huangzhengjie/hyperopt/train_xlab.npy', x_tr)
np.save('/data1/huangzhengjie/hyperopt/test_xlab.npy', x_te)